In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [2]:
# import our chat-bot intents file
import json

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")


45 documents


In [4]:
# classes = intents
print (len(classes), "classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [5]:
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-6-72b6ba6ac079>:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [8]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [9]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 1s 1ms/step - loss: 2.2830 - accuracy: 0.2145
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 2.2063 - accuracy: 0.1205
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0744 - accuracy: 0.2269
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1201 - accuracy: 0.2784
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 1.9119 - accuracy: 0.4801
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 1.8935 - accuracy: 0.3724
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 1.6766 - accuracy: 0.4779
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 1.6849 - accuracy: 0.5311
Epoch 9/200
9/9 [==============================] - 0s 2ms/step - loss: 1.5758 - accuracy: 0.5883
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 1.3207 - accuracy: 0.6467
Epoch 11/200
9/9 [===========

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [11]:
p = bow("Load blood pessure for patient", words)

print(p)
print(classes)

found in bag: load
found in bag: blood
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [12]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[6.8438015e-07 5.1901125e-06 9.9983633e-01 6.1983570e-07 5.2803404e-07
  1.5473158e-04 1.6355931e-06 2.6904502e-07 9.5887053e-08]]


In [13]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [14]:
classify_local('Hello, good day!')

found in bag: hello
found in bag: ,
found in bag: good
found in bag: day


[('greeting', '0.99998236')]

In [15]:
classify_local('How you can assist me?')

found in bag: how
found in bag: you
found in bag: can
found in bag: me


[('options', '0.99935716')]

In [16]:
classify_local('Get me to adverse medicine form')

found in bag: me
found in bag: to
found in bag: advers


[('adverse_drug', '0.99767095')]

In [17]:
classify_local('Place to log blood pressure')

found in bag: to
found in bag: log
found in bag: blood
found in bag: press


[('blood_pressure', '0.99999046')]

In [18]:
classify_local('Fetch blood result for patient')

found in bag: blood
found in bag: result
found in bag: for
found in bag: paty


[('blood_pressure_search', '0.99999213')]

In [19]:
classify_local('Blood pressure monitoring in hospital')

found in bag: blood
found in bag: press
found in bag: hospit


[('blood_pressure', '0.8727928')]

In [20]:
classify_local('Look for hospital to monitor blood pressure')

found in bag: look
found in bag: for
found in bag: hospit
found in bag: to
found in bag: blood
found in bag: press


[('hospital_search', '0.9342201')]

In [21]:
model.save('saved_model/chatbot-brains-model')

INFO:tensorflow:Assets written to: saved_model/chatbot-brains-model\assets
